In [ ]:
"""

Look at the distribution of the weights to make a figure out of it

"""

In [ ]:
import xarray as xr
import numpy as np
from tqdm.notebook import tqdm
import seaborn as sns
import multimelt.useful_functions as uf
import os
#from xhistogram.xarray import histogram
import scipy.io
import matplotlib.pyplot as plt


In [ ]:
sns.set_context('paper')

In [ ]:
%matplotlib qt5

READ IN DATA

In [ ]:
home_path = '/bettik/burgardc/'
plot_path = '/bettik/burgardc/PLOTS/summer_paper_plots/'
outputpath_GL = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/GL_FLUX/'
outputpath_weights = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/ANALYSIS/'
inputpath_raw = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/'
inputpath_data=home_path+'/DATA/SUMMER_PAPER/interim/'
inputpath_weights = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/ANALYSIS/'


In [ ]:
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)

In [ ]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

In [ ]:
BedMachine_orig = xr.open_dataset(inputpath_data+'BedMachine_v2_aggregated4km_allvars.nc')
file_BedMachine = uf.cut_domain_stereo(BedMachine_orig, map_lim, map_lim)
file_isf_conc = file_BedMachine['isf_conc']

grid_cell_area_file = xr.open_dataset(inputpath_data+'gridarea_ISMIP6_AIS_4000m_grid.nc').sel(x=file_isf.x,y=file_isf.y)
true_grid_cell_area = grid_cell_area_file['cell_area'].drop('lon').drop('lat')
cell_area_weight = true_grid_cell_area/(4000 * 4000)

lon = file_isf.longitude
lat = file_isf.latitude

xx = file_isf.x
yy = file_isf.y
dx = (xx[2] - xx[1]).values
dy = (yy[2] - yy[1]).values
grid_cell_area_const = abs(dx*dy)  
grid_cell_area_weighted = file_isf_conc * grid_cell_area_const * cell_area_weight

isf_stack_mask = uf.create_stacked_mask(file_isf['ISF_mask'], file_isf.Nisf, ['y','x'], 'mask_coord')


In [ ]:
sorted_isf_rignot = [11,69,43,28,12,57,
                     70,44,29,13,58,71,45,30,14,
                     59,72,46,
                     31,
                     15,61,73,47,32,16,48,33,17,62,49,34,18,63,74,
                     50,35,19,64,
                     10,
                     36,20,65,51,37,
                     22,38,52,23,66,53,39,24,
                     67,40,54,75,25,41,
                     26,42,55,68,60,27]

In [ ]:
weight_2300_file = xr.open_dataset(inputpath_weights + 'bayesian_weights_davison_varying_combined_2300_withoutGISS.nc')

In [ ]:
bay_weights = weight_2300_file['bay_weights']

In [ ]:
sens_weights = xr.DataArray(data=np.array([0.11,
                                           0.24,
                                           0.03,
                                           0.10,
                                           0.10,
                                           0.10,
                                           0.10,
                                           0.24,
                                           0.47,
                                           0.41,
                                           0.12,
                                           0.43,
                                           0.39,
                                           0.05]), dims=['model']).assign_coords({'model': 
                                                                                  ['ACCESS-CM2','ACCESS-ESM1-5','CanESM5',
                                                                                   'CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
                                                                                   'GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 'IPSL-CM6A-LR',
                                                                                   'MPI-ESM1-2-HR','MRI-ESM2-0','UKESM1-0-LL']})

In [ ]:
model_2300 = ['ACCESS-CM2','ACCESS-ESM1-5','CanESM5','CESM2-WACCM', 'IPSL-CM6A-LR','MRI-ESM2-0','UKESM1-0-LL'] #,'GISS-E2-1-H'

In [ ]:
ddim = 'm' #'param','model','m'


f = plt.figure()
f.set_size_inches(8.25*1.2, 8.25*1.2)

ax={}

leg_hdl = []

i = 0

weight = bay_weights * sens_weights
weight['param'] = ['linear','quadratic Ant slope','quadratic local slope','plume','box','neural network']
weight['m'] = ['1','3','5']

for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    weight_norm_kisf = weight.sel(Nisf=kisf, model=model_2300) / weight.sel(Nisf=kisf).sum(['param','model','m'])

    if ddim == 'param':
        weight_ddim = weight_norm_kisf.stack(stack_dim=['model','m'])
    elif ddim == 'model':
        weight_ddim = weight_norm_kisf.stack(stack_dim=['param','m'])
    elif ddim == 'm':        
        weight_ddim = weight_norm_kisf.stack(stack_dim=['param','model'])

    bins = weight_ddim[ddim]
    _, bin_edges = np.histogram(weight_ddim.values, bins=np.arange(len(bins)), weights=weight_ddim.values)

    hist = weight_ddim.sum('stack_dim') # / weight_ddim.sum()

    ax[i].bar(weight_ddim[ddim], hist,  edgecolor='None')#width=1,

    #ax[i].set_title(ano_choice.sel(Nisf=kisf))

    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:#
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values))
    #if file_isf['isf_name'].sel(Nisf=kisf).values not in ['Larsen B', 'Wordie']:
    #    ax[i].set_ylim(0,mass_balance_simu_anoISMIP.sel(Nisf=kisf).max().values)
    #ax[i].axvline(x=30, c='k', linestyle='--')
    
    if i < 56:
        ax[i].set_xticklabels('')
    if i not in [0,8,16,24,32,40,48,56]:
        ax[i].set_yticklabels('')
        
    i = i+1

    plt.xticks(rotation=90)
#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
f.savefig(plot_path + 'histo_weights_per_iceshelf_2300_'+ddim+'.pdf')

In [ ]:
weight.sel(model=model_2300).stack(stack_dim=['model','m','Nisf']).sum('stack_dim')

In [ ]:
f = plt.figure()
f.set_size_inches(8.25/2, 8.25/2)
    
#weight_norm_kisf = weight.sel(model=model_2300) / weight.sum(['param','model','m'])

hist = weight.sel(model=model_2300).sum(['Nisf','model','m'])
plt.bar(weight['param'], hist,  edgecolor='None')
plt.xticks(rotation=90)
f.tight_layout()
sns.despine()
f.savefig(plot_path + 'histo_weights_2300_param_summedallisf.pdf')

In [ ]:
weight